In [ ]:
from pyspark.sql import SparkSession

from delta import configure_spark_with_delta_pip

builder = SparkSession.builder.appName("MyApp") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")

spark = configure_spark_with_delta_pip(builder).getOrCreate()

In [ ]:
data = spark.range(0, 5)
data.write.format("delta").save("/data/delta-table")

In [ ]:
!ls /data/delta-table

In [ ]:
!ls /data/delta-table/_delta_log

In [ ]:
!cat /data/delta-table/_delta_log/00000000000000000000.json

In [ ]:
df = spark.read.format("delta").load("/tmp/delta-table")
df.show()